In [1]:
import pandas as pd
sub_data = pd.read_csv('subscribers.csv')

In [2]:
data1 = sub_data[['package_type','preferred_genre', 'intended_use', 'weekly_consumption_hour',
                          'age', 'male_TF', 'plan_type','monthly_price', 'join_fee', 
                  'attribution_technical','attribution_survey', 'op_sys', 'discount_price','payment_period',
                 'cancel_date','current_sub_TF']]
data1 = data1[data1.age<100]
data1 = data1[data1.age>10]
data1 = data1[data1.weekly_consumption_hour>0]
#data1.dropna(inplace=True)

In [3]:
import numpy as np
import math
conditions = [
    (data1['payment_period'] == 0) & (data1['current_sub_TF'] == False),
    (data1['payment_period'] == 0) & (data1['current_sub_TF'] == True) & (data1['cancel_date'].notna()),
    (data1['payment_period'] == 0) & (data1['current_sub_TF'] == True) & (data1['cancel_date'].isnull()),
    ]

# create a list of the values we want to assign for each condition
values = [1,1,np.nan]

# create a new column and use np.select to assign values to it using our lists as arguments
data1['churn'] = np.select(conditions, values)
data1.describe()

,weekly_consumption_hour,age,monthly_price,join_fee,discount_price,payment_period,churn
count,188718.000000,188718.000000,188718.000000,188718.000000,188718.000000,188718.000000,182316.000000
mean,28.007279,46.142639,4.734884,0.115269,4.514929,0.768575,0.417884
std,4.949419,13.954908,0.015787,0.177169,0.021783,0.783617,0.493212
min,1.202235,16.000000,4.000300,-0.660600,3.780100,0.000000,0.000000
25%,24.401536,35.000000,4.734300,0.036700,4.514100,0.000000,0.000000
50%,27.301448,46.000000,4.734300,0.036700,4.514100,1.000000,0.000000
75%,30.201361,57.000000,4.734300,0.110100,4.514100,1.000000,1.000000
max,76.599962,99.000000,5.101300,0.734000,5.027900,3.000000,1.000000


In [4]:
data1.dropna(inplace=True)
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19157 entries, 9 to 227627
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   package_type             19157 non-null  object 
 1   preferred_genre          19157 non-null  object 
 2   intended_use             19157 non-null  object 
 3   weekly_consumption_hour  19157 non-null  float64
 4   age                      19157 non-null  float64
 5   male_TF                  19157 non-null  object 
 6   plan_type                19157 non-null  object 
 7   monthly_price            19157 non-null  float64
 8   join_fee                 19157 non-null  float64
 9   attribution_technical    19157 non-null  object 
 10  attribution_survey       19157 non-null  object 
 11  op_sys                   19157 non-null  object 
 12  discount_price           19157 non-null  float64
 13  payment_period           19157 non-null  int64  
 14  cancel_date          

In [5]:
data2 = pd.get_dummies(data=data1, columns=['package_type','preferred_genre', 
                                                      'intended_use',
                                                       'male_TF', 'attribution_technical',
                                                      'attribution_survey',
                                                      'op_sys','plan_type'])


In [6]:
data2= data2.drop(columns=['payment_period','cancel_date','current_sub_TF'],axis=1)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
         data2.drop(columns=['churn'],axis=1), data2['churn'], test_size=0.3, random_state=47)

In [8]:
import xgboost as xgb
from sklearn import metrics 
xgb_model = xgb.XGBClassifier(random_state=47)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
y_prob = xgb_model.predict_proba(X_test)[:,1]
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("AUC:",metrics.roc_auc_score(y_test, y_prob))
print(metrics.classification_report(y_test,y_pred))

Accuracy: 0.7738343771746694
AUC: 0.8085586306191009
              precision    recall  f1-score   support

         0.0       0.52      0.34      0.41      1330
         1.0       0.82      0.91      0.86      4418

    accuracy                           0.77      5748
   macro avg       0.67      0.62      0.63      5748
weighted avg       0.75      0.77      0.76      5748



In [9]:
from sklearn.linear_model import LogisticRegression
lg_model = LogisticRegression(random_state=47)
lg_model.fit(X_train, y_train)
y_pred = lg_model.predict(X_test)
y_prob = lg_model.predict_proba(X_test)[:,1]
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("AUC:",metrics.roc_auc_score(y_test, y_prob))
print(metrics.classification_report(y_test,y_pred))

Accuracy: 0.7807933194154488
AUC: 0.8075133510553204
              precision    recall  f1-score   support

         0.0       0.55      0.30      0.39      1330
         1.0       0.82      0.92      0.87      4418

    accuracy                           0.78      5748
   macro avg       0.68      0.61      0.63      5748
weighted avg       0.75      0.78      0.76      5748



/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
